## Importing Libraries

In [ ]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
from prettytable import PrettyTable
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay, classification_report
plt.rcParams["font.weight"] = "bold"
plt.rcParams["axes.labelweight"] = "bold"

In [ ]:
DATASETS_FOLDER = "C:/Users/Killian/Desktop/FYP-Multistage-Throughput-Predictor/Datasets/Final_Outputs/"
IMAGES_OUTPUT_FOLDER = "C:/Users/Killian/Desktop/Paper V2/Images/"
TEST_FOLDER = "C:/Users/Killian/Desktop/FYP-Multistage-Throughput-Predictor/Datasets/Testing/"

## Defining Analysis Functions

In [ ]:
def import_data(model_name="univariate_baseline", just_preds=False, mb=True):
    if just_preds:
        predicted = np.load(DATASETS_FOLDER+model_name+"_predicted_y.npy")
        if mb:
            return predicted/1024
        return predicted
    true = np.squeeze(np.load(DATASETS_FOLDER+model_name+"_true_y.npy"))
    predicted = np.load(DATASETS_FOLDER+model_name+"_predicted_y.npy")
    if mb:
        return true/1024, predicted/1024
    return true, predicted

In [ ]:
def get_variance(data):
    return np.var(data)

In [ ]:
def get_std(data):
    return np.std(data)

In [ ]:
def get_ts_residuals(true, pred):
        residuals = true-pred
        return residuals

In [ ]:
import seaborn as sns
def time_step_boxplots(residuals=[], model_names=[], title="", xlim=(), savefig_path="", xlab="Residuals (Mbps)",showfliers = True):
    df = pd.DataFrame()
    for model, residuals in zip(model_names, residuals):
        for i in range(residuals.shape[1]):
            temp_df = pd.DataFrame()
            temp_df[xlab] = residuals[:,i]
            temp_df["Horizon (seconds)"] = i+1
            temp_df["Model"] = model
            df = pd.concat([df, temp_df])
    sns.set_theme(style="whitegrid")
    if showfliers:
        box = sns.boxplot(x=xlab, y="Horizon (seconds)",
                    hue="Model",
                    data=df, orient="h", whis=(5,95), saturation=1, flierprops={'marker': 'o', 'markersize': 5, 'markerfacecolor': 'pink'})
    else:
        box = sns.boxplot(x=xlab, y="Horizon (seconds)",
            hue="Model",
            data=df, orient="h", whis=(5,95), saturation=1, showfliers=False)
    if xlim:
        box.set_xlim(xlim)
    box.set_title(title)
    plt.legend(bbox_to_anchor=(1.02, 1), loc='upper left', borderaxespad=0)
    # box.set_ylabel("Horizon (seconds)")
    if savefig_path:
        plt.savefig(savefig_path, dpi=300, bbox_inches='tight')
    plt.show() # Display the plot on the screen
    

In [ ]:
def get_mape(true, predicted, epsilon=50/1024):
    denominator = np.squeeze(true) + epsilon
    try:
        mape = np.mean(np.abs((np.squeeze(true) - predicted)/denominator))*100
    except Exception as e:
        print(e)
        mape = "n/a"
    return mape

In [ ]:
def get_ape(true, predicted, epsilon=50/1024):
    denominator = np.squeeze(true) + epsilon
    try:
        ape = np.abs((np.squeeze(true) - predicted)/denominator)*100
    except Exception as e:
        print(e)
        ape = "n/a"
    return ape

In [ ]:
def get_mse(true, predicted):
    mse = np.mean(np.power(np.squeeze(true)-predicted, 2))
    return mse

In [ ]:
def get_mae(true, predicted):
    mae = np.mean(np.abs(np.squeeze(true)-predicted))
    return mae

In [ ]:
def create_latex_table(table):
    latex_string = "\\begin{table}[!htb]\n\\centering\n\\begin{tabular}{|c|c|c|c|c|c|}\n\\hline\n{Model} & {Mean Resids (Mbps)} & {Resids std (Mbps)} & {MAPE} & {MSE (Mbps)} & {MAE (Mbps)}\\\\\n\\hline\n"
    for row in table:
        for i in range(len(row)):
            if i == len(row) - 1:
                latex_string += str(row[i]) + "\\\\\n\\hline\n"
            else:
                latex_string += str(row[i]) + " & "
    latex_string_end = "\\end{tabular}\n\\label{tab:train_test_dist}\n\\end{table}"
    latex_string = latex_string + latex_string_end
    print(latex_string)

In [ ]:
def carry_out_analysis(prefix="univariate", title_prefix="Univariate"):
    for dataset in ["low", "medium", "high", 1]:
        if dataset == 1:
            base_true, base_pred = import_data(model_name="{}_baseline".format(prefix))
            multiOne_true, multiOne_pred = import_data(model_name="{}_multiOne".format(prefix))
            multiAll_true, multiAll_pred = import_data(model_name="{}_multiAll".format(prefix))

            # Get Residuals
            base_resids = get_ts_residuals(base_true, base_pred)
            multiOne_resids = get_ts_residuals(multiOne_true, multiOne_pred)
            multiAll_resids = get_ts_residuals(multiAll_true, multiAll_pred)

            # Get Absolute Percent Error
            base_ape = get_ape(base_true, base_pred)
            multiOne_ape = get_ape(multiOne_true, multiOne_pred)
            multiAll_ape = get_ape(multiAll_true, multiAll_pred)
            
            # Boxplot of residuals
            time_step_boxplots(residuals=[base_resids, multiOne_resids, multiAll_resids],
                                model_names=["Baseline", "Multistage One", "Multistage All"],
                                  title="Model Performance on All Test Sequences",
                                  savefig_path=IMAGES_OUTPUT_FOLDER+"{} On All Test Data".format(title_prefix))
            
            # Boxplot of Absolute Percent Error
            time_step_boxplots(residuals=[base_ape, multiOne_ape, multiAll_ape],
                                model_names=["Baseline", "Multistage One", "Multistage All"],
                                  title="Absolute Percent Error on All Test Sequences",
                                  savefig_path=IMAGES_OUTPUT_FOLDER+"APE of {} On All Test Data".format(title_prefix), xlab="Absolute Percent Error")
            
            # Boxplot of residuals No outliers
            time_step_boxplots(residuals=[base_resids, multiOne_resids, multiAll_resids],
                                model_names=["Baseline", "Multistage One", "Multistage All"],
                                  title="Model Performance on All Test Sequences",
                                  savefig_path=IMAGES_OUTPUT_FOLDER+"{} On All Test Data no Outliers".format(title_prefix), showfliers = False)
            
            # Boxplot of Absolute Percent Error no Outliers
            time_step_boxplots(residuals=[base_ape, multiOne_ape, multiAll_ape],
                                model_names=["Baseline", "Multistage One", "Multistage All"],
                                  title="Absolute Percent Error on All Test Sequences",
                                  savefig_path=IMAGES_OUTPUT_FOLDER+"APE of {} On All Test Data no Outliers".format(title_prefix), xlab="Absolute Percent Error", showfliers = False)


            # Standard Dev
            base_std = get_std(base_resids)
            multiOne_std = get_std(multiOne_resids)
            multiAll_std = get_std(multiAll_resids)

            # MAPE
            base_mape = get_mape(base_true, base_pred)
            multiOne_mape = get_mape(multiOne_true, multiOne_pred)
            multiAll_mape = get_mape(multiAll_true, multiAll_pred)

            # MSE
            base_mse = get_mse(base_true, base_pred)
            multiOne_mse = get_mse(multiOne_true, multiOne_pred)
            multiAll_mse = get_mse(multiAll_true, multiAll_pred)

            # MAE
            base_mae = get_mae(base_true, base_pred)
            multiOne_mae = get_mae(multiOne_true, multiOne_pred)
            multiAll_mae = get_mae(multiAll_true, multiAll_pred)

            # Table of metrics
            table = PrettyTable(["Model", "Mean Resids (Mbps)", " Resids std (Mbps)", "MAPE", "MSE", "MAE"])
            table.add_row(["Baseline",np.mean(base_resids), base_std, base_mape, base_mse, base_mae])
            table.add_row(["Multistage One",np.mean(multiOne_resids), multiOne_std, multiOne_mape, multiOne_mse, multiOne_mae])
            table.add_row(["Multistage All",np.mean(multiAll_resids), multiAll_std, multiAll_mape, multiAll_mse, multiAll_mae])
            print("Residuals of All Sequences Test Set")
            #print(table)
            raw_table = []
            raw_table.append(["Baseline",round(np.mean(base_resids),3), round(base_std,3), round(base_mape,3), round(base_mse,3), round(base_mae,3)])
            raw_table.append(["Multistage One",round(np.mean(multiOne_resids),3), round(multiOne_std,3), round(multiOne_mape,3), round(multiOne_mse,3), round(multiOne_mae,3)])
            raw_table.append(["Multistage All",round(np.mean(multiAll_resids),3), round(multiAll_std,3), round(multiAll_mape,3), round(multiAll_mse,3), round(multiAll_mae,3)])
            create_latex_table(raw_table)
        else:
            base_true, base_pred = import_data(model_name="{}_baseline_{}".format(prefix, dataset))
            single_true, single_pred = import_data(model_name="{}_multiOne_{}".format(prefix, dataset))

            # Get Residuals
            base_resids = get_ts_residuals(base_true, base_pred)
            single_resids = get_ts_residuals(single_true, single_pred)

            # Get APE
            base_ape = get_ape(base_true, base_pred)
            single_ape = get_ape(single_true, single_pred)
            
            # Boxplot of residuals
            time_step_boxplots(residuals=[base_resids, single_resids],
                                model_names=["Baseline", "{} Only".format(dataset)],
                                  title="Model Performance on {} Test Sequences".format(dataset.capitalize()),
                                  savefig_path=IMAGES_OUTPUT_FOLDER+"{} On {} Test Set".format(title_prefix, dataset))

            # Boxplot of APE
            time_step_boxplots(residuals=[base_ape, single_ape],
                                model_names=["Baseline", "{} Only".format(dataset)],
                                  title="Absolute Percent Error on {} Test Sequences".format(dataset.capitalize()),
                                  savefig_path=IMAGES_OUTPUT_FOLDER+"APE of {} On {} Test Set".format(title_prefix, dataset), xlab="Absolute Percent Error")
            
            # Boxplot of residuals no Outliers
            time_step_boxplots(residuals=[base_resids, single_resids],
                                model_names=["Baseline", "{} Only".format(dataset)],
                                  title="Model Performance on {} Test Sequences".format(dataset.capitalize()),
                                  savefig_path=IMAGES_OUTPUT_FOLDER+"{} On {} Test Set no Outliers".format(title_prefix, dataset), showfliers = False)

            # Boxplot of APE no Outliers
            time_step_boxplots(residuals=[base_ape, single_ape],
                                model_names=["Baseline", "{} Only".format(dataset)],
                                  title="Absolute Percent Error on {} Test Sequences".format(dataset.capitalize()),
                                  savefig_path=IMAGES_OUTPUT_FOLDER+"APE of {} On {} Test Set no Outliers".format(title_prefix, dataset), xlab="Absolute Percent Error", showfliers = False)

          ###############
            multiOne_pred = import_data(model_name="{}_multiOne_ms_{}".format(prefix, dataset), just_preds=True)
            multiAll_pred = import_data(model_name="{}_multiAll_ms_{}".format(prefix, dataset), just_preds=True)

            #resids
            multiOne_resids = get_ts_residuals(single_true, multiOne_pred)
            multiAll_resids = get_ts_residuals(single_true, multiAll_pred)

            multiOne_ape = get_ape(single_true, multiOne_pred)
            multiAll_ape = get_ape(single_true, multiAll_pred)

            # Boxplot of residuals
            time_step_boxplots(residuals=[base_resids, multiOne_resids, multiAll_resids],
                                model_names=["Baseline", "Multistage One", "Multistage All"],
                                  title="Model Performance on {} Test Sequences".format(dataset.capitalize()),
                                  savefig_path=IMAGES_OUTPUT_FOLDER+"MS {} On {} Test Set".format(title_prefix, dataset))

            # Boxplot of APE
            time_step_boxplots(residuals=[base_ape, multiOne_ape, multiAll_ape],
                                model_names=["Baseline", "Multistage One", "Multistage All"],
                                  title="Absolute Percent Error on {} Test Sequences".format(dataset.capitalize()),
                                  savefig_path=IMAGES_OUTPUT_FOLDER+"MS APE of {} On {} Test Set".format(title_prefix, dataset), xlab="Absolute Percent Error")
            
            # Boxplot of residuals no Outliers
            time_step_boxplots(residuals=[base_resids, multiOne_resids, multiAll_resids],
                                model_names=["Baseline", "Multistage One", "Multistage All"],
                                  title="Model Performance on {} Test Sequences".format(dataset.capitalize()),
                                  savefig_path=IMAGES_OUTPUT_FOLDER+"MS {} On {} Test Set no Outliers".format(title_prefix, dataset), showfliers = False)

            # Boxplot of APE no Outliers
            time_step_boxplots(residuals=[base_ape, multiOne_ape, multiAll_ape],
                                model_names=["Baseline", "Multistage One", "Multistage All"],
                                  title="Absolute Percent Error on {} Test Sequences".format(dataset.capitalize()),
                                  savefig_path=IMAGES_OUTPUT_FOLDER+"MS APE of {} On {} Test Set no Outliers".format(title_prefix, dataset), xlab="Absolute Percent Error", showfliers = False)
                        # Standard Dev
            base_std = get_std(base_resids)
            multiOne_std = get_std(multiOne_resids)
            multiAll_std = get_std(multiAll_resids)

            # MAPE
            base_mape = get_mape(base_true, base_pred)
            multiOne_mape = get_mape(single_true, multiOne_pred)
            multiAll_mape = get_mape(single_true, multiAll_pred)

            # MSE
            base_mse = get_mse(base_true, base_pred)
            multiOne_mse = get_mse(single_true, multiOne_pred)
            multiAll_mse = get_mse(single_true, multiAll_pred)

            # MAE
            base_mae = get_mae(base_true, base_pred)
            multiOne_mae = get_mae(single_true, multiOne_pred)
            multiAll_mae = get_mae(single_true, multiAll_pred)
            
            
            raw_table = []
            raw_table.append(["Baseline",round(np.mean(base_resids),3), round(base_std,3), round(base_mape,3), round(base_mse,3), round(base_mae,3)])
            raw_table.append(["Multistage One",round(np.mean(multiOne_resids),3), round(multiOne_std,3), round(multiOne_mape,3), round(multiOne_mse,3), round(multiOne_mae,3)])
            raw_table.append(["Multistage All",round(np.mean(multiAll_resids),3), round(multiAll_std,3), round(multiAll_mape,3), round(multiAll_mse,3), round(multiAll_mae,3)])
            print("\n================\nMULTISTAGE\n===============\n")
            create_latex_table(raw_table)
            print("\n================\n")
          #####################

            # Standard Dev
            base_std = get_std(base_resids)
            single_std = get_std(single_resids)

            # MAPE
            base_mape = get_mape(base_true, base_pred)
            single_mape = get_mape(single_true, single_pred)

            # MSE
            base_mse = get_mse(base_true, base_pred)
            single_mse = get_mse(single_true, single_pred)

            # MAE
            base_mae = get_mae(base_true, base_pred)
            single_mae = get_mae(single_true, single_pred)

            # Table of metrics
            table = PrettyTable(["Model", "Mean Resids (Mbps)", " Resids std (Mbps)", "MAPE", "MSE", "MAE"])
            table.add_row(["Baseline",np.mean(base_resids), base_std, base_mape, base_mse, base_mae])
            table.add_row(["{} Only".format(dataset),np.mean(single_resids), single_std, single_mape, single_mse, single_mae])
            print("Residuals of {} Sequences Test Set".format(dataset))
            #print(table)
            
            # Latex
            raw_table = []
            raw_table.append(["Baseline",round(np.mean(base_resids),3), round(base_std,3), round(base_mape,3), round(base_mse,3), round(base_mae,3)])
            raw_table.append(["{} Only".format(dataset),round(np.mean(single_resids),3), round(single_std,3), round(single_mape,3), round(single_mse,3), round(single_mae,3)])
            create_latex_table(raw_table)

    true, pred = import_data(model_name="{}_multiOne_classifier".format(prefix))
    true = np.argmax(true, axis=-1)
    pred = np.argmax(pred, axis=-1)
    conf_matrix = confusion_matrix(true, pred)
    # create a dictionary to map class names
    class_names = {0: "low", 1: "medium", 2: "high"}

    # map the class names
    true_names = [class_names[x] for x in true]
    pred_names = [class_names[x] for x in pred]
    report = classification_report(true_names, pred_names, labels=["low", "medium", "high"])
    print("Classification Report:\n",report)
    display_conf = ConfusionMatrixDisplay(conf_matrix, display_labels=["Low", "Medium", "High"])
    display_conf.plot(cmap="Oranges")
    plt.savefig(IMAGES_OUTPUT_FOLDER+"{} Confusion Matrix".format(title_prefix), dpi=300, bbox_inches='tight')
    plt.show()

## Analysis

In [ ]:
carry_out_analysis("constraint_1_5", "Size Constraint of 1 and a half MB")

In [ ]:
carry_out_analysis("constraint_3", "Constraint 3Mb")

In [ ]:
carry_out_analysis("univariate", "Univariate")

In [ ]:
carry_out_analysis("Multivariate", "Multivariate")

# Classifier Error Analysis

### Univariate

In [ ]:
true, pred = import_data(model_name="univariate_multiAll_classifier")
x = np.load(TEST_FOLDER+"univariate_test_x.npy").squeeze()

In [ ]:
x.shape

In [ ]:
create_csv(x, true, pred)

In [ ]:
true, pred = import_data(model_name="univariate_multiAll_classifier")
true = np.argmax(true, axis=-1)
pred = np.argmax(pred, axis=-1)
conf_matrix = confusion_matrix(true, pred)
# create a dictionary to map class names
class_names = {0: "low", 1: "medium", 2: "high"}
# map the class names
true_names = [class_names[x] for x in true]
pred_names = [class_names[x] for x in pred]
report = classification_report(true_names, pred_names, labels=["low", "medium", "high"])
print("Classification Report:\n",report)

In [ ]:
x = np.load("C:/Users/Killian/Desktop/FYP-Multistage-Throughput-Predictor/Datasets/Testing/univariate_classifier_test_x.npy")
y = np.load("C:/Users/Killian/Desktop/FYP-Multistage-Throughput-Predictor/Datasets/Testing/univariate_test_y.npy")

In [ ]:
df = pd.DataFrame()
df["true"] = true_names
df["pred"] = pred_names

In [ ]:
print(conf_matrix)

In [ ]:
low_as_med = df[(df["true"]=="low") & (df["pred"]=="medium")].index.to_list()
med_as_low = df[(df["true"]=="medium") & (df["pred"]=="low")].index.to_list()
low_med_boundary = low_as_med + med_as_low

In [ ]:
tp_low_med_boundary = y[low_med_boundary,:]
tp_low_med_boundary = tp_low_med_boundary/1024
low_med_bound_means = np.mean(tp_low_med_boundary, axis=1)
low_med_bound_std = np.std(tp_low_med_boundary, axis=1)
mean_val = [np.mean(low_med_bound_means)]*(low_med_bound_means.shape[0])
plt.plot(low_med_bound_means)
plt.plot(mean_val, color="red")

In [ ]:
mean_val = [np.mean(low_med_bound_means)]*(low_med_bound_means.shape[0])
plt.plot(low_med_bound_means)
plt.plot(mean_val, color="red")

In [ ]:
plt.plot(low_med_bound_means)
plt.plot(mean_val, color="red")

In [ ]:
low_bound = y[low_as_med,:]/1024
low_mean = np.mean(low_bound)
low_std = np.std(low_bound)
mean_vector = [low_mean]*low_bound.shape[0]
std_vector = [low_std]*low_bound.shape[0]
std_vector_up = [low_std+low_mean]*low_bound.shape[0]
std_vector_down = [low_mean-low_std]*low_bound.shape[0]
plt.ylim(0,1.4)
plt.plot([1]*low_bound.shape[0], linestyle="-", color="blue")
plt.plot(mean_vector, color="red")
plt.plot(std_vector_up, linestyle="--", color="red")
plt.plot(std_vector_down, linestyle="--", color="red")

In [ ]:
med_bound = y[med_as_low,:]/1024
med_mean = np.mean(med_bound)
med_std = np.std(med_bound)
mean_vector = [med_mean]*med_bound.shape[0]
std_vector = [med_std]*med_bound.shape[0]
std_vector_up = [med_std+med_mean]*med_bound.shape[0]
std_vector_down = [med_mean-med_std]*med_bound.shape[0]
#plt.ylim(0,1.4)
plt.plot([1]*med_bound.shape[0], linestyle="-", color="blue")
plt.plot(mean_vector, color="red")
plt.plot(std_vector_up, linestyle="--", color="red")
plt.plot(std_vector_down, linestyle="--", color="red")